# Two Tower Passage Ranking applyed to MS MARCO
### Desenvolvido por Graziella Bonadia e Matheus Sasso

## Hyperparâmetros

In [ ]:
#@title Configurações gerais
experiment_name = 'TwoTowerPassageRanking'  #@param {type:"string"}
model_name = 'bert-base-uncased'  #@param ["bert-base-uncased","bert-large-uncased","albert-base-v2"] {type:"string"}
batch_size =  4#@param {type:"integer"}
max_epochs = 3 #@param {type:"integer"}
accumulate_grad_batches = 16  #@param {type:"integer"}
max_length = 100  #@param {type:"integer"}
learning_rate = 5e-3  #@param {type:"number"}

In [ ]:
#Transformers
from transformers import BertModel
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
hyperparams = {'model':model,'tokenizer':tokenizer,'learning_rate':learning_rate,'batch_size':batch_size,'max_length':max_length,'accumulate_grad_batches':accumulate_grad_batches} 

Se eu estiver LOCAL

In [33]:
checkup_dataframe_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Vetores Densos\\notebooks\\checkup_dataframe.zip'
checkup_dataframe_dir = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Vetores Densos\\notebooks'
model_checkpoint_path = 'C:\\Users\\Matheus\\Desktop\\Pós Graduação\\NLP\\Projeto\\Vetores Densos\\checkpoint docTTTTTquery\\model.ckpt-1004000'

## Instalações Externas

In [10]:
# ! pip install pytorch-lightning  --quiet
# ! pip install transformers  --quiet

Para a utilizaçõ de TPU

In [ ]:
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

## Imports

In [11]:
# Importar todos os pacotes de uma só vez para evitar duplicados ao longo do notebook.

# General Libs
import gzip
import os
import random
import pdb

#Hardware Data
from multiprocessing import cpu_count
import psutil
import GPUtil as GPU
GPUs = GPU.getGPUs()

# #Drive
# from google.colab import drive

#Data Sciens Libs
import numpy as np
import pandas as pd

#Lightning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

#Torch
import torch
from torch import Tensor
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn.functional as F

# TORCH XLA - TPU
import torch_xla.core.xla_model as xm

#Generic Types
from typing import Dict
from typing import List
from typing import Tuple


## Configurações Gerais

Definição de variáveis Globais


In [0]:
# Configurações gerais
accumulate_grad_batches = 16
# Important: Fix seeds so we can replicate results
seed = 123
random.seed(seed)
# np.random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)

DICA para modelos reais: Um modelo otimizado deve manter o uso de GPU próximo a 100% durante o treino.
Vamos utilizar a bilioteca abaixo para monitorar isso. Note que no modelo simples utilizado aqui o uso não vai chegar a 100%.

In [18]:
print(GPUs[0].name)
dev = "cuda" if torch.cuda.is_available() else "cpu"
print(dev)
device = torch.device(dev)

GeForce 920M
cpu


In [19]:
print(f"Pytorch Lightning Version: {pl.__version__}")

def hardware_stats():
    '''
    Returns a dict containing some hardware related stats
    '''

    return {"cpu": str(psutil.cpu_percent()) + '%',
            "mem": str(psutil.virtual_memory().percent) + '%',
            "gpu": str([gpu.name for gpu in GPUs]) + '%'}

# print(f"Imports loaded succesfully. Current GPU: {torch.cuda.get_device_name(0)}, number of CPU cores: {cpu_count()}")
print(f"Imports loaded succesfully.  number of CPU cores: {cpu_count()}")

Pytorch Lightning Version: 0.7.6
Imports loaded succesfully.  number of CPU cores: 4


Quadro da NVIDIA

In [20]:
# !nvidia-smi

Decorator para impedir quebra de memório recorrente

In [21]:
#Impedir quebra de memória 
# https://docs.fast.ai/troubleshoot.html#memory-leakage-on-exception
import functools, traceback
def gpu_mem_restore(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = sys.exc_info()
            traceback.clear_frames(tb)
            raise type(val).with_traceback(tb) from None
    return wrapper

Iremos salvar os checkpoints (pesos do modelo) no google drive, para que possamos continuar o treino de onde paramos.

In [22]:
# drive.mount('/content/drive')

## Preparando Dados

Download do dataset de dev 

In [29]:
!wget https://git.uwaterloo.ca/jimmylin/anserini-indexes/raw/master/index-msmarco-passage-20191117-0ed488.tar.gz
!tar xvfz index-msmarco-passage-20191117-0ed488.tar.gz

'wget' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.
tar: Error opening archive: Failed to open 'index-msmarco-passage-20191117-0ed488.tar.gz'


## Carregando o dataset

Criaremos uma divisão de treino (100k pares) e val (5k pares) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [ ]:
import zipfile
with zipfile.ZipFile(checkup_dataframe_path, 'r') as zip_ref:
    zip_ref.extractall(checkup_dataframe_dir)

In [ ]:
df_queries  = pd.read_csv(checkup_dataframe_dir + 'checkup_dataframe.csv') 
df_queries.head()

In [0]:
tokenizer.encode_plus('we like pizza',
                      max_length =10,
                      pad_to_max_length=True)

{'attention_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [62, 114, 6871, 0, 0, 0, 0, 0, 0, 0],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

Criando Dataset


In [0]:
class MyDataset(Dataset):
    def __init__(self, text: List[Tuple[str]], tokenizer, max_length: int = 100,training_step=False):
        self.tokenizer = tokenizer
        self.text = text
        self.max_length = max_length
        self.training_step = training_step
        
    def __len__(self):
        return len(self.text_pairs)
    
    def __getitem__(self, idx):
        text = self.text.iloc[idx]
        original_query = text['ORIGINAL_QUERY']
        positive_query =  text['POSITIVE_QUERY']
        negative_query =  text['NEGATIVE_QUERY']
 
        # Original Query
        original_query_encode = f"{original_query} {tokenizer.eos_token}"
        original_query_encode = self.encoder_plus(original_query_encode,self.max_length)
        original_query_token_ids = original_query_encode['input_ids']
        original_query_mask = original_query_encode['attention_mask']
        original_query_token_ids = torch.tensor(original_query_token_ids).type(torch.long)
        original_query_mask = torch.tensor(original_query_mask).type(torch.long)
                               
                               
        # Positonal Query
        positive_query_encode = f"{positive_query} {tokenizer.eos_token}"
        positive_query_encode = self.encoder_plus(original_query_encode,self.max_length)
        positive_query_token_ids = positive_query_encode['input_ids']
        positive_query_mask = positive_query_encode['attention_mask']
        positive_query_token_ids = torch.tensor(positive_query_token_ids).type(torch.long)
        positive_query_mask = torch.tensor(positive_query_mask).type(torch.long)
                               
        
        # Negative Query
        negative_query_encode = f"{negative_query} {tokenizer.eos_token}"
        negative_query_encode = self.encoder_plus(original_query_encode,self.max_length)
        negative_query_token_ids = negative_query_encode['input_ids']
        negative_query_mask = negative_query_encode['attention_mask']
        negative_query_token_ids = torch.tensor(negative_query_token_ids).type(torch.long)
        negative_query_mask = torch.tensor(negative_query_mask).type(torch.long)
        
        
        return (original_query_token_ids,original_query_mask,positive_query_token_ids,
                positive_query_mask,negative_query_token_ids,negative_query_mask,
                original_query,positive_query,negative_query)
    
    def encoder_plus(self,text,L):
      return self.tokenizer.encode_plus(text,
                                        max_length = L,
                                        pad_to_max_length=True)
    

## Testando o DataLoader

In [0]:
text = df_train[:10]

print(text)

print(50*'-')

dataset_debug = MyDataset(
    text=text,
    tokenizer=hyperparams['tokenizer'],
    max_length=hyperparams['max_length'])

dataloader_debug = DataLoader(dataset_debug, batch_size=10, shuffle=True,num_workers=cpu_count())

original_query_token_ids,original_query_mask,positive_query_token_ids,
positive_query_mask,negative_query_token_ids,negative_query_mask,
original_query,positive_query,negative_query = next(iter(dataloader_debug))

        
print('original_query_token_ids:\n', original_query_token_ids)
print('original_query_mask:\n', original_query_mask)
print('positive_query_token_ids:\n', positive_query_token_ids)
print('positive_query_mask:\n', positive_query_mask)
print('negative_query_token_ids:\n', negative_query_token_ids)
print('negative_query_mask:\n', negative_query_mask)


print('original_query_token_ids.query:\n', original_query_token_ids.query)
print('original_query_mask.query:\n', original_query_mask.query)
print('positive_query_token_ids.query:\n', positive_query_token_ids.query)
print('positive_query_mask.query:\n', positive_query_mask.query)
print('negative_query_token_ids.query:\n', negative_query_token_ids.query)
print('negative_query_mask.query:\n', negative_query_mask.query)


print(original_query)
print(positive_query)
print(negative_query)

source_token_ids:
 tensor([[13959,  1566,    12, 21076,    10,    62,   114,  6871,     1,     0,
             0,     0,     0,     0,     0]])
source_mask:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])
target_token_ids:
 tensor([[   3,   15,   76,  281,    7,  235,   20, 6871,    1,    0]])
target_mask:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])
Source Word:
 we like pizza
Target Word:
 eu gosto de pizza
source_token_ids.shape: torch.Size([1, 15])
source_mask.shape: torch.Size([1, 15])
target_token_ids.shape: torch.Size([1, 10])
target_mask.shape: torch.Size([1, 10])


## Loss Funtion example 

In [ ]:
def loss_document_passages():
    '''
    Implamentation
    '''

## FAISS example

https://github.com/facebookresearch/faiss/wiki/Getting-started

Getting some data

In [ ]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

Building an index and adding the vectors to it

In [ ]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

Searching

In [ ]:
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:]) 

## Criando o Two Tower Passage Ranking com Pytorch Lightning

In [0]:
class TwoTowerFineTunwer(pl.LightningModule):

    def __init__(self, 
                 hyperparams,
                 all_data,
                 use_tpu = False,
                 overfit=False):
      
        super(TwoTowerFineTunwer, self).__init__()
        
        
        #---------- Definição do modelo
        tower1 = BertModel.from_pretrained('bert-base-uncased')
        tower2 = BertModel.from_pretrained('bert-base-uncased')
        

        #---------- Hyperparametros
        self.tokenizer = hyperparams['tokenizer']
        self.learning_rate = hyperparams['learning_reate']
        self.batch_size = hyperparams['batch_size']
        self.source_max_length = hyperparams['max_length']
        self.overfit = overfit


        #---------- Carregamento datasets (Para eu poder variar self.max_length)
        if overfit:
          self.train_dataset = MyDataset(all_data[0], tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
          self.valid_dataset = MyDataset(all_data[0], tokenizer=self.tokenizer, source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
          self.test_dataset =   MyDataset(all_data[0], tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
        else:
          self.train_dataset = MyDataset(all_data[0], tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
          self.valid_dataset = MyDataset(all_data[1], tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
          self.test_dataset = MyDataset(all_data[2],  tokenizer=self.tokenizer,source_max_length = self.source_max_length, target_max_length =  self.target_max_length)
        
        
        #---------- Loss Function
        self.loss_funtion = criterion
            

        #---------- Algorythim step
        self.training = True

        

    def forward(self, source_token_ids, source_mask, target_token_ids=None,
                target_mask=None):
        
        # outputs[0] --> Valor bruto, shape []
        # outputs[1] -->  shape [B,L,32128]
        # outputs[2] -->  shape [B,36,512]

        if self.training:
            outputs_tower1 = self.tower1(input_ids=source_token_ids,attention_mask=source_mask,lm_labels=target_token_ids)
            outputs_tower2 = self.tower2(input_ids=source_token_ids,attention_mask=source_mask,lm_labels=target_token_ids)
            
            loss = loss_document_passages(outputs_tower1,outputs_tower2)# for explícito no mini-batch
            
            return loss
        else:    
            
            # FAISS ONE QUERY
            
            
            # FAISS SEARCH
            D, I = index.search(xq, k) 
           
        
        
            return nucleous_sampling_output

    def training_step(self, batch, batch_nb):
        # batch
        source_token_ids, source_mask, target_token_ids, target_mask, _, _ = batch
         
        # fwd
        self.training = True
        loss = self.forward(source_token_ids, source_mask, target_token_ids)

        # What to log
        tensorboard_logs = {'loss': loss}

        # Stats to be printed on TQDM bar
        tqdm_dict = hardware_stats()


        return {'loss': loss, 'log': tensorboard_logs, 'progress_bar': tqdm_dict}

    def validation_step(self, batch, batch_nb):
           
            

        #ISSO AQUI DA ESTRANHO, EU DEVERIA PEGAR PARA CADA VALOR DO BATCH E NÂO PARA A POSIÇÂO 0

        # batch
        self.training = False
        source_token_ids, source_mask, target_token_ids, target_mask, original_source, original_target = batch
        original_source, original_target = original_source[0], original_target[0]# Quando a string passa pelo data loader ele coloca como tupla: ('exemplo',)

        # scoring
        nucleous_sampling_output = self.forward(source_token_ids, source_mask)# Ja são os ids mas entre dois [[]]
        generated_target = self.tokenizer.decode(nucleous_sampling_output[0], skip_special_tokens=False)
        bleu_score = sacrebleu.corpus_bleu([generated_target], [[original_target]]).score
        
        #Adicionando elementos ao dataframe
        self.df_valid = self.df_valid.append(pd.Series([original_source,original_target,generated_target,bleu_score], index=self.df_valid.columns ), ignore_index=True)


        # Stats to be printed on TQDM bar
        tqdm_dict = hardware_stats()

        return {'val_bleu': bleu_score, 'progress_bar': tqdm_dict}

    def test_step(self, batch, batch_nb):
        # batch
        self.training = False
        source_token_ids, source_mask, target_token_ids, target_mask, original_source, original_target = batch
        original_source, original_target = original_source[0], original_target[0]# Quando a string passa pelo data loader ele coloca como tupla: ('exemplo',)
       
        # scoring
        nucleous_sampling_output = self.forward(source_token_ids, source_mask)
        generated_target = self.tokenizer.decode(nucleous_sampling_output[0], skip_special_tokens=False)
        bleu_score = sacrebleu.corpus_bleu([generated_target], [[original_target]]).score
        
        #Adicionando elementos ao dataframe
        self.df_test = self.df_test.append(pd.Series([original_source,original_target,generated_target,bleu_score], index=self.df_test.columns),ignore_index=True)

        # Stats to be printed on TQDM bar
        tqdm_dict = hardware_stats()

        return {'test_bleu': bleu_score, 'progress_bar': tqdm_dict}

    def validation_epoch_end(self, outputs):
        avg_bleu = sum([x['val_bleu'] for x in outputs]) / len(outputs)

        tensorboard_logs = {'avg_val_bleu': avg_bleu}
        
        return {'avg_val_bleu': avg_bleu, 'progress_bar': tensorboard_logs}

    def test_epoch_end(self, outputs):
        avg_bleu = sum([x['test_bleu'] for x in outputs]) / len(outputs)

        tensorboard_logs = {'avg_test_bleu': avg_bleu}
        
        return {'avg_test_bleu': avg_bleu, 'progress_bar': tensorboard_logs}
    
    def configure_optimizers(self):
        return torch.optim.AdamW(
            [p for p in self.parameters() if p.requires_grad],
            lr=self.learning_rate, eps=1e-08)
    
    
    def check_sampler(dataset):
        #---------- TPU Use
        if this.use_tpu:
            sampler = None
        else: 
            sampler = torch.utils.data.distributed.DistributedSampler(
                dataset,
                num_replicas=xm.xrt_world_size(),
                rank=xm.get_ordinal(),
                shuffle=True
            )
        return sampler

    @gpu_mem_restore
    def train_dataloader(self):
        shuffle = False if self.overfit else True
        sampler = check_sampler(self.train_dataset)
        return DataLoader(self.train_dataset,sampler, batch_size=self.batch_size, shuffle=shuffle,num_workers=cpu_count())
    
    @gpu_mem_restore
    def val_dataloader(self):
        sampler = check_sampler(self.valid_dataset)
            #FAISS INDEX
            #EP para todas as passagens offline
            index = faiss.IndexFlatL2(d)   # build the index
            print(index.is_trained)
            index.add(xb)
        return DataLoader(self.valid_dataset,sampler, batch_size=self.batch_size, shuffle=False,num_workers=cpu_count())
    
    @gpu_mem_restore
    def test_dataloader(self):
        sampler = check_sampler(self.test_dataset)
           #FAISS INDEX
            #EP para todas as passagens offline
            index = faiss.IndexFlatL2(d)   # build the index
            print(index.is_trained)
            index.add(xb)
        return DataLoader(self.test_dataset, sampler,batch_size=self.batch_size,shuffle=False, num_workers=cpu_count())

## Número de parâmetros do modelo

In [0]:
all_data = [ds_train,ds_val,ds_test]

In [0]:
model = T5Finetuner(hparams, all_data) 

In [0]:
sum([torch.tensor(x.size()).prod() for x in model.parameters() if x.requires_grad]) # trainable parameters

tensor(60506880)

## Testando rapidamente o modelo em treino, validação e teste com um batch

In [0]:
trainer = pl.Trainer(gpus=1, 
                     checkpoint_callback=False,  # Disable checkpoint saving.
                     fast_dev_run=True)
trainer.fit(model)
trainer.test(model)
# Para não ter estouro de mémoria da GPU

INFO:lightning:Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:
    | Name                                                                  | Type                       | Params
-----------------------------------------------------------------------------------------------------------------
0   | model                                                                 | T5ForConditionalGeneration | 60 M  
1   | model.shared                                                          | Embedding                  | 16 M  
2   | model.encoder                                                         | T5Stack                    | 35 M  
3   | model.encoder.block                                                   | ModuleList                 | 18 M  
4   | model.encoder.block.0                                                 | T5Block                    

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_bleu': 0.0}
--------------------------------------------------------------------------------



In [0]:
del model

## Overfit em algumas amostras


Antes de treinar o modelo no dataset todo, faremos overfit do 
modelo em poucas de treino para verificar se loss vai para próximo de 0. Isso serve para depurar se a implementação do modelo está correta.

Podemos também medir se a acurácia neste minibatch chega perto de 100%. Isso serve para depurar se nossa função que mede a acurácia está correta.

Nota: se treinarmos por muitas épocas (ex: 500) é possivel que a loss vá para zero mesmo com bugs na implementação. O ideal é que a loss chege próxima a zero antes de 100 épocas.

In [0]:
trainer = pl.Trainer(gpus=1,
                     max_epochs=30,
                     check_val_every_n_epoch=10,
                     checkpoint_callback=False,  # Disable checkpoint saving
                     overfit_pct=0.005,
                     profiler=True,
                     accumulate_grad_batches=accumulate_grad_batches)
all_data = [ds_train]
model = T5Finetuner(hparams, all_data,overfit=True )
trainer.fit(model)


INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:
    | Name                                                                  | Type                       | Params
-----------------------------------------------------------------------------------------------------------------
0   | model                                                                 | T5ForConditionalGeneration | 60 M  
1   | model.shared                                                          | Embedding                  | 16 M  
2   | model.encoder                                                         | T5Stack                    | 35 M  
3   | model.encoder.block                                                   | ModuleList                 | 18 M  
4   | model.encoder.block.0                                                 | T5Block                    | 3 M   
5   | model.encoder.block.0.layer                                           | ModuleList          

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


INFO:lightning:

Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_train_start      	|  0.027183       	|  0.027183       
on_epoch_start      	|  0.0028499      	|  0.085497       
get_train_batch     	|  0.0073753      	|  13.939         
on_batch_start      	|  2.3386e-05     	|  0.043499       
model_forward       	|  0.043691       	|  81.265         
model_backward      	|  0.14442        	|  268.62         
on_after_backward   	|  3.3294e-06     	|  0.0061927      
on_batch_end        	|  0.0032431      	|  6.0321         
optimizer_step      	|  0.017758       	|  16.515         
on_epoch_end        	|  1.7534e-05     	|  0.00052601     
on_train_end        	|  0.001308       	|  0.001308       



1

In [0]:
trainer.test(model)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_bleu': 64.05501862497174}
--------------------------------------------------------------------------------



In [0]:
del model  # Para não ter estouro de mémoria da GPU

## Treinamento e Validação no dataset todo

Salvamento parcial (Israel)

In [0]:
n_steps = len(ds_train) # how many training steps will happen
save_times = 1000
save_every = n_steps // save_times
save_every

100

In [0]:
max_epochs = 1

checkpoint_path = '/content/drive/My Drive/Mestrado/PLN/Solucoes_Entregues/checkpoint_save/lab9/checkpoints_epocas/epoch=0.ckpt'
checkpoint_dir = os.path.dirname(os.path.abspath(checkpoint_path))
print(f'Files in {checkpoint_dir}: {os.listdir(checkpoint_dir)}')
print(f'Saving checkpoints to {checkpoint_dir}')
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_dir,
                                      save_top_k=-1,
                                      monitor="val_acc",
                                      mode="max")  # Keeps all checkpoints.


resume_from_checkpoint = None
if os.path.exists(checkpoint_path):
    print(f'Restoring checkpoint: {checkpoint_path}')
    resume_from_checkpoint = checkpoint_path

all_data = [ds_train,ds_val,ds_test]
trainer = pl.Trainer(gpus=1,
                     max_epochs=max_epochs,
                     check_val_every_n_epoch=1,
                     profiler=True,
                     accumulate_grad_batches=accumulate_grad_batches,
                     checkpoint_callback=checkpoint_callback,
                     progress_bar_refresh_rate=30,
                     resume_from_checkpoint=resume_from_checkpoint)

model = T5Finetuner(hparams,
                    all_data,
                    save_every=save_every,
                    save_to='/content/drive/My Drive/Mestrado/PLN/Solucoes_Entregues/checkpoint_save/lab9/checkpoints_parciais')

trainer.fit(model)

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]


Files in /content/drive/My Drive/Mestrado/PLN/Solucoes_Entregues/checkpoint_save/lab9/checkpoints_epocas: ['epoch=0.ckpt']
Saving checkpoints to /content/drive/My Drive/Mestrado/PLN/Solucoes_Entregues/checkpoint_save/lab9/checkpoints_epocas
Restoring checkpoint: /content/drive/My Drive/Mestrado/PLN/Solucoes_Entregues/checkpoint_save/lab9/checkpoints_epocas/epoch=0.ckpt


INFO:lightning:
    | Name                                                                  | Type                       | Params
-----------------------------------------------------------------------------------------------------------------
0   | model                                                                 | T5ForConditionalGeneration | 60 M  
1   | model.shared                                                          | Embedding                  | 16 M  
2   | model.encoder                                                         | T5Stack                    | 35 M  
3   | model.encoder.block                                                   | ModuleList                 | 18 M  
4   | model.encoder.block.0                                                 | T5Block                    | 3 M   
5   | model.encoder.block.0.layer                                           | ModuleList                 | 3 M   
6   | model.encoder.block.0.layer.0                                     

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


INFO:lightning:

Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_train_start      	|  0.033313       	|  0.033313       
on_train_end        	|  0.001498       	|  0.001498       



1

## Após treinado, avaliamos o modelo no dataset de teste

É importante que essa avaliação seja feita poucas vezes para evitar "overfit manual" no dataset de teste.

In [0]:
trainer.test(model)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_bleu': 19.734138788503238}
--------------------------------------------------------------------------------



#Salvando os resultados em um .csv

In [0]:
compression_opts = dict(method='zip', archive_name='enpt_t5_results.csv') 
model.df_test.to_csv('enpt_t5_results.zip', index=False,compression=compression_opts)  

## Conclusão

* Nucleous sampling não é bom para tradução, recomendado a usar greedy search ou beam search
* Vale a pena treinar até 3 épocas
* Usar o truque dos ascentos do Alexandre


# Fim do Notebook